# Bayesian Networks in PyMC

A **Bayesian Network** (also called *Bayes Net* or *Belief Network*) is a probabilistic graphical model that represents a set of variables and their conditional dependencies via a directed acyclic graph (DAG). Each node in the DAG corresponds to a random variable, and edges represent conditional dependencies.

## Key Ideas

1. **Directed Acyclic Graph (DAG)**  
   Each variable is a node. Edges indicate direct dependencies. There are no directed cycles.

2. **Conditional Probability Distributions (CPDs)**  
   Each node has a conditional distribution \(P(X \mid \text{parents}(X))\).

3. **Chain Rule for Bayesian Networks**  
   The joint probability factorizes as  
   \[
   P(X_1, X_2, \ldots, X_n) \;=\; \prod_{i=1}^{n} P(X_i \,\mid\, \text{parents}(X_i))
   \]

4. **Inference**  
   Inference involves computing the posterior distribution of a set of variables given evidence (observations). Exact inference can be done via message passing in small networks, but for more complex networks, sampling-based (Monte Carlo) methods or variational methods are often used.

---

## Example: “Sprinkler–Rain–Wet Grass”

A classic toy Bayesian Network has three random variables:

1. **Rain (R)**: Indicates whether it is raining.  
2. **Sprinkler (S)**: Indicates whether the garden sprinkler is turned on.  
3. **Wet Grass (W)**: Indicates whether the grass is wet.

The dependencies are as follows:

- The chance of the sprinkler being on depends on whether it’s raining.  
- The grass is wet if either it rains or the sprinkler is on (or both).

Graphically:



### Step 1: Define the CPDs

Let’s define each node’s conditional probability distribution (CPD) as follows:

1. **P(Rain)**  
   \[
   P(R = \text{true}) = 0.2, \quad P(R = \text{false}) = 0.8.
   \]

2. **P(Sprinkler | Rain)**  
   - If it is raining:  
     \[
       P(S = \text{true} \mid R = \text{true}) = 0.1
     \]  
   - If it is not raining:  
     \[
       P(S = \text{true} \mid R = \text{false}) = 0.5
     \]

3. **P(Wet Grass | Rain, Sprinkler)**  
   - For simplicity:  
     \[
       P(W = \text{true} \mid R = \text{true}, S = \text{true}) = 0.99
     \]
     \[
       P(W = \text{true} \mid R = \text{true}, S = \text{false}) = 0.9
     \]
     \[
       P(W = \text{true} \mid R = \text{false}, S = \text{true}) = 0.8
     \]
     \[
       P(W = \text{true} \mid R = \text{false}, S = \text{false}) = 0.0
     \]


## Step 2: Implementing in PyMC

We'll use [PyMC](https://github.com/pymc-devs/pymc), a Python library for probabilistic programming, to represent and sample from this Bayesian network. PyMC can handle discrete and continuous variables, as well as more complex models.

### Example Code

Below is an example of how to define and sample from the "Sprinkler–Rain–Wet Grass" network using **PyMC 4+**.


In [ ]:
import pymc as pm
import aesara.tensor as at

# Define the model
with pm.Model() as model:
    # 1. Rain (Bernoulli)
    #    P(Rain=True) = 0.2
    p_rain = 0.2
    rain = pm.Bernoulli("rain", p=p_rain)
    
    # 2. Sprinkler depends on Rain
    #    P(Sprinkler=True | Rain=True)  = 0.1
    #    P(Sprinkler=True | Rain=False) = 0.5

    def sprinkler_p(r):
        return 0.1 * r + 0.5 * (1 - r)
    
    sprinkler = pm.Bernoulli(
        "sprinkler",
        p=sprinkler_p(rain)
    )

    # 3. Wet Grass depends on Rain and Sprinkler
    #    P(W=true | R=true,  S=true)  = 0.99
    #    P(W=true | R=true,  S=false) = 0.90
    #    P(W=true | R=false, S=true)  = 0.80
    #    P(W=true | R=false, S=false) = 0.00
    
    # Lookup table for W=true:
    w_true_probs = at.as_tensor_variable([
        [0.0, 0.8],   # R = false (0), columns: S=false(0), S=true(1)
        [0.9, 0.99]   # R = true  (1), columns: S=false(0), S=true(1)
    ])
    
    p_wet_grass = w_true_probs[rain, sprinkler]
    wet_grass = pm.Bernoulli("wet_grass", p=p_wet_grass)

    # Sampling
    trace = pm.sample(draws=2000, tune=1000, chains=2, random_seed=42)


### Step 3: Inspecting Results

After sampling, we can summarize the trace to see the distribution of each variable:



In [ ]:
pm.summary(trace, var_names=["rain", "sprinkler", "wet_grass"])


This will give you statistics (mean, standard deviation, highest posterior density intervals, etc.) for each random variable, based on the sampled chain.

---

## Interpretation

- We have defined three Bernoulli variables: **Rain**, **Sprinkler**, and **Wet Grass**.
- We used a simple table or function to encode the conditional probabilities.
- PyMC’s sampling (`pm.sample()`) method generates samples from the joint (if unobserved) or posterior (if some variables are observed).
- From the resulting trace, you can estimate quantities like:
  \[
    P(\text{rain} = \text{true} \mid \text{wet_grass} = \text{true})
  \]
  by filtering the samples where `wet_grass=True` and checking the proportion of those samples for which `rain=True`.

---

## Summary

- A **Bayesian Network** models how variables depend on each other via conditional probabilities in a DAG.  
- **PyMC** can represent and sample from both discrete and continuous Bayesian networks.  
- This notebook demonstrates a small discrete Bayesian network (“Sprinkler–Rain–Wet Grass”) and how to perform sampling-based inference in PyMC.

